### Purpose

This notebook acquires a short time series and then a z stack at a set interval of time for a given duration.

In this example we acquire a time series of `duration = 2` seconds and then a z stack parameterized by
```z_start = -2.5; z_end = 2.5; z_step = 0.25``` every `subset_interval = 20` seconds for `total_duration = 60` seconds. 
That is, we acquire data subsets containing a 2 second time series and a 5 micrometer z stack every 20 seconds over the
span of 60 seconds.

Using `pycro-manager` this set of acquisition events is encoded as follows:

```
events = []
for s in range(num_subsets):
    for t in range(num_time_points):
        events.append({'axes': {'subset':s, 'time':t, 'z':0},
                       'z': z_sequence[0],
                       'min_start_time': s*subset_interval})
    for z in range(num_z_slices):
        events.append({'axes': {'subset':s, 'time':num_time_points, 'z': z},
                       'z': z_sequence[z],
                       'min_start_time': s*subset_interval})
```

In [ ]:
from pycromanager import Bridge
from pycromanager import Acquisition, multi_d_acquisition_events

import numpy as np

### Construct java objects

In [8]:
bridge = Bridge()
mmc = bridge.get_core()
mmStudio = bridge.get_studio()

### Set acquisition parameters

In [9]:
# Data set parameters
path = r'C:\test'
name = 'pycromanager test'
total_duration = 60  # in seconds
subset_interval = 20  # in seconds

# z stack parameters
z_start = -2.5
z_end = 2.5
z_step = 0.25
relative = True
sequence = False

# time series parameters
duration = 2  # in seconds
framerate = 10

num_subsets = np.ceil(total_duration / subset_interval).astype(np.int)
num_time_points = duration * framerate
z_sequence = np.arange(z_start,z_end+z_step,z_step)
num_z_slices = len(z_sequence)

### Prepare for acquisition

In [10]:
# setup cameras
mmc.set_property('Camera', 'Framerate', framerate)

# setup z stage
z_stage = mmc.get_focus_device()

if relative:
    z_pos = mmc.get_position(z_stage)

    z_sequence += z_pos

if sequence:
    mmc.set_property(z_stage,'UseSequence','Yes')

### Generate events

In [11]:
events = []
for s in range(num_subsets):
    for t in range(num_time_points):
        events.append({'axes': {'subset':s, 'time':t, 'z':0},
                       'z': z_sequence[0],
                       'min_start_time': s*subset_interval})
    for z in range(num_z_slices):
        events.append({'axes': {'subset':s, 'time':num_time_points, 'z': z},
                       'z': z_sequence[z],
                       'min_start_time': s*subset_interval})

### Acquire data

In [12]:
if __name__ == '__main__':
    with Acquisition(directory=path, name=name) as acq:
        acq.acquire(events)